# Value-based agent in the cart-pole environment using Apple MLX

|        | TYPE             | VALUES | DESCRIPTION                                                                     |
|--------|------------------|--------|---------------------------------------------------------------------------------|
| Action | ndarray<br/>(1,) | {0, 1} | Direction to push cart.<br/>0: left<br/>1: right                                |
| Observation | ndarray<br/>(4,) | float  | 1. Cart position<br/>2. Cart velocity<br/>3. Pole angle<br/>4. Angular velocity |
| Reward |  | float | Reward is given for every step taken, including the termination step.<br/>Each step provides a reward of +1 |
| Termination &<br/>Truncation |  | boolean | Pole Angle exceeds ± 0.2095 rad<br/>Cart Position exceeds ± 2.4 (i.e., edge of the display)<br/>Episode length is greater than 500 |

In [ ]:
import time
import imageio
import mlx.core as mx
import gymnasium as gym
import mlx.optimizers as optim
import matplotlib.pyplot as plt

from DQN import DeepQ, Network

In [ ]:
environment = gym.make('CartPole-v1', render_mode="rgb_array")

### Training

#### Parameters

In [ ]:
GAMES = 2000

GAMMA = 0.99

EXPLORATION_RATE = 1.0
EXPLORATION_DECAY = 0.995
EXPLORATION_MIN = 0.01

MINIBATCH = 64
TRAIN_EVERY = 10

MEMORY = 500
RESET_Q_EVERY = 250

NETWORK = {"inputs": 4, "outputs": 2, "nodes": [30, 15]}
OPTIMIZER = {"optimizer": optim.RMSprop, "lr": 0.0025}

#### Initialisation

In [ ]:
value_agent = DeepQ(
    network=NETWORK, optimizer=OPTIMIZER,
    gamma=GAMMA, batch_size=MINIBATCH, memory=MEMORY,
    exploration_rate=EXPLORATION_RATE, exploration_decay=EXPLORATION_DECAY, 
    exploration_min=EXPLORATION_MIN
)

# TARGET NETWORK
# --------------------------------------------------------------------------------------------------
# The target network is used to calculate the target Q-value for the loss function. It is updated
# every `RESET_Q_EVERY` games to the current Q-network. The parameters are then updated in the 
# same way as here:

_value_agent = Network(network=NETWORK)

parameters = [(f"layers.{i}.{name}", value)
              for i, layer in enumerate(value_agent.agent.parameters()["layers"])
              for name, value in layer.items()]
_value_agent.load_weights(parameters)

In [ ]:
checkpoint = GAMES // 10
metrics = {
    "steps": mx.zeros(GAMES),
    "losses": mx.zeros(GAMES // TRAIN_EVERY),
    "exploration": mx.zeros(GAMES)
}

#### Training

In [ ]:
start = time.time()
for game in range(1, GAMES + 1):
    
    state = mx.array(environment.reset()[0])
    terminated = truncated = False
    
    # LEARNING FROM GAME
    # ----------------------------------------------------------------------------------------------
    
    steps = 0
    while not (terminated or truncated):
        steps += 1
        
        action = value_agent.action(state)
        new_state, reward, terminated, truncated, _ = environment.step(action.item())
        new_state = mx.array(new_state)
        
        value_agent.remember(state, action, new_state, mx.array([reward]))
        state = new_state
    value_agent.memorize(steps)
    
    if game % TRAIN_EVERY == 0:
        loss = value_agent.learn(network=_value_agent)
        metrics["losses"][game // TRAIN_EVERY - 1] = loss
    
    if game % RESET_Q_EVERY == 0:
        parameters = [(f"layers.{i}.{name}", value)
                      for i, layer in enumerate(value_agent.agent.parameters()["layers"])
                      for name, value in layer.items()]
        _value_agent.load_weights(parameters)

    # METRICS
    # ----------------------------------------------------------------------------------------------
    
    metrics["steps"][game-1] = steps
    metrics["exploration"][game-1] = value_agent.explore["rate"]
    
    if game % checkpoint == 0 or game == GAMES:
        _mean_steps = metrics["steps"][max(0, game-checkpoint-1):game-1].mean().item()
        _mean_loss = (metrics["losses"][max(0, (game-checkpoint-1) 
                                            // TRAIN_EVERY):game // TRAIN_EVERY].mean().item())
        
        print(f"Game {game:>6} {int(game/GAMES * 100):>16} % \n"
              f"{'-'*30} \n"
              f" > Average steps: {int(_mean_steps):>12} \n"
              f" > Average loss: {_mean_loss:>13.4f} \n ")
        
print(f"Total training time: {time.time()-start:.2f} seconds")

#### Visualisation

##### Metrics

In [ ]:
def moving_average(data, window_size=50):
    """Compute moving average with given window size of the data."""
    half_window = window_size // 2
    return [(data[max(0, i-half_window):min(len(data), i+half_window)]).mean().item() 
            for i in range(len(data))]

steps = moving_average(metrics["steps"])
losses = moving_average(metrics["losses"])

fig, ax = plt.subplots(2, 1, figsize=(12, 8))
fig.suptitle("Value-based: deep Q-learning agent")

ax[0].axhline(y=500, color="gray", linestyle="dotted", linewidth=1)
ax[0].plot(steps, color="black", linewidth=1)
ax[0].set_xticks([])
ax[0].set_title("Average steps per game")

ax[1].plot(mx.linspace(0, GAMES, len(losses)).tolist(), losses, color="black", linewidth=1)
ax[1].set_yscale("log")
ax[1].set_xlabel("Game nr.")
ax[1].set_title("Average loss")

ax_2 = ax[1].twinx()
ax_2.plot(metrics["exploration"].tolist(), color="gray", linewidth=0.5)
ax_2.set_ylabel("Exploration rate")
ax_2.yaxis.label.set_color('gray')
ax_2.tick_params(axis='y', colors='gray')

for i in range(0, GAMES, GAMES // 10):
    ax[0].axvline(x=i, color='gray', linewidth=0.5)
    ax[1].axvline(x=i, color='gray', linewidth=0.5)

plt.savefig("../output/mlx-value-cartpole.png")
plt.show()

##### In action

In [ ]:
state = mx.array(environment.reset()[0])

images = []
terminated = truncated = False
while not (terminated or truncated):
    actions = value_agent.agent(state)
    action = mx.argmax(actions).item()
    
    state, reward, terminated, truncated, _ = environment.step(action)
    state = mx.array(state)

    images.append(environment.render())
_ = imageio.mimsave('../output/mlx-value-cartpole.gif', images, duration=25)

<img src="../output/mlx-value-cartpole.gif" width="1000" height="1000" />

In [ ]:
environment.close()